In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import io
import time
import re
from datetime import datetime
from typing import List, Set

In [3]:
# --- CONFIGURAÇÕES ---
# O ano mínimo para começar a coletar demonstrações financeiras
START_YEAR = 2015 
# O número de dias úteis para calcular o retorno anual (aproximadamente 252)
TRADING_DAYS_IN_YEAR = 252

# --- 1.1. Coletando Tickers Históricos e Atuais do S&P 500 ---
print("--- 1.1. Coletando lista de Tickers (Históricos e Atuais) do S&P 500... ---")

all_tickers = set()

try:
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    tables = pd.read_html(response.text)
    
    # 1. Tabela de Empresas Atuais
    df_current = tables[0]
    # Limpa e adiciona símbolos
    all_tickers.update(df_current["Symbol"].str.replace(r'\.', '-', regex=True).tolist())
    
    # 2. Tabela de Mudanças (geralmente a última)
    if len(tables) > 1:
        df_changes = tables[-1]
        for col in df_changes.columns:
            if "Added" in str(col) or "Removed" in str(col):
                new_tickers = df_changes[col].dropna().astype(str).str.replace(r'\.', '-', regex=True).tolist()
                all_tickers.update(new_tickers)

    # Limpar e Padronizar Tickers (remove caracteres inválidos, limita a 5 caracteres para evitar lixo)
    tickers = sorted([t.replace("-", ".") if t in ["BRK.B", "BF.B"] else t.replace(".", "-") for t in all_tickers if t and len(t) < 6 and not t.startswith('^')])
    print(f"✅ Lista inicial (bruta) de {len(tickers)} tickers obtida da Wikipedia.")
    
except Exception as e:
    print(f"❌ Erro ao obter tickers da Wikipedia ({e}). Usando lista de fallback para validação.")
    tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'JPM', 'JNJ', 'WMT', 'XOM', 'V']

# --- 1.2. Validação dos Tickers via YFinance ---
validated_tickers = []
print(f"\n--- 1.2. Validando {len(tickers)} Tickers via YFinance (Pode levar tempo para a lista completa) ---")

for i, ticker in enumerate(tickers):
    try:
        # Tenta obter a informação básica e verifica se tem preço de mercado
        info = yf.Ticker(ticker).info
        if info.get('regularMarketPrice') is not None and info.get('regularMarketPrice') > 0 and info.get('quoteType') == 'EQUITY':
            validated_tickers.append(ticker)
        # else: # Ticker existe, mas não é uma ação negociada ativamente (ex: fundos, índice)
        #     pass 
    except Exception:
        # print(f"❌ {ticker} falhou na validação.")
        pass
    time.sleep(0.05) # Pequeno delay

tickers = validated_tickers
df_tickers = pd.DataFrame(tickers, columns=['Ticker'])
df_tickers.to_csv("sp500_tickers_historicos_validos.csv", index=False)
print(f"\n✅ Validação concluída. {len(tickers)} tickers válidos salvos em 'sp500_tickers_historicos_validos.csv'.")

--- 1.1. Coletando lista de Tickers (Históricos e Atuais) do S&P 500... ---


C:\Users\15495612.USUARIOS\AppData\Local\Temp\ipykernel_13268\3222194644.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


✅ Lista inicial (bruta) de 893 tickers obtida da Wikipedia.

--- 1.2. Validando 893 Tickers via YFinance (Pode levar tempo para a lista completa) ---


HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ANSS"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: AETNA"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ALCOA"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: AMCOR"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: AMOCO"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ANSYS"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: AVAYA"}}}
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found


✅ Validação concluída. 637 tickers válidos salvos em 'sp500_tickers_historicos_validos.csv'.


In [9]:
def get_yfinance_raw_columns(tickers: List[str]) -> Set[str]:
    """
    Coleta os nomes de todas as colunas brutas (itens financeiros)
    retornadas pelas demonstrações anuais do yfinance para uma lista de tickers.
    """
    
    all_raw_columns = set()
    
    # Função auxiliar para limpar os nomes das colunas (retirar espaços e caracteres)
    def clean_column_name(col_name: str) -> str:
        # Garante que estamos lidando com strings antes de usar re.sub
        if isinstance(col_name, str):
            # Remove caracteres que não sejam letras, números ou underscore
            return re.sub(r'[^A-Za-z0-9_]+', '', col_name)
        return "" # Retorna string vazia para não-strings (deve ser o caso apenas de datas, que já corrigimos)

    print(f"--- Iniciando a inspeção das colunas para {len(tickers)} tickers ---")

    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)

            # 1. Balanço Patrimonial (Balance Sheet)
            if not stock.balance_sheet.empty:
                # CORREÇÃO: Usar .index para obter os nomes dos itens, não as datas
                bs_cols = stock.balance_sheet.index.tolist()
                all_raw_columns.update(bs_cols)
            
            # 2. Demonstração do Resultado (Income Statement)
            if not stock.financials.empty:
                # CORREÇÃO: Usar .index para obter os nomes dos itens, não as datas
                is_cols = stock.financials.index.tolist()
                all_raw_columns.update(is_cols)
            
            # 3. Fluxo de Caixa (Cash Flow)
            if not stock.cashflow.empty:
                # CORREÇÃO: Usar .index para obter os nomes dos itens, não as datas
                cf_cols = stock.cashflow.index.tolist()
                all_raw_columns.update(cf_cols)
            
        except Exception as e:
            # Captura erro de ticker não encontrado ou erro de conexão
            print(f"⚠️ Aviso: Falha ao inspecionar {ticker}: {e}")

    # Limpa os nomes das colunas como é feito no Bloco 2 para facilitar a comparação
    # A set comprehension agora opera sobre strings, resolvendo o TypeError.
    cleaned_columns = {clean_column_name(col) for col in all_raw_columns}
    
    # Remove strings vazias resultantes da limpeza
    cleaned_columns.discard("") 
    
    return sorted(list(cleaned_columns))

# Tickers para inspeção (uma amostra robusta de diferentes setores)
SAMPLE_TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'JPM', 'XOM', 'TSLA', 'JNJ', 'WMT', 'V']

# Executa a função
unique_cleaned_columns = get_yfinance_raw_columns(SAMPLE_TICKERS)

print("\n--- Nomes ÚNICOS e LIMPOS das Colunas do YFinance Encontradas: ---")
print(f"Total de itens financeiros únicos: {len(unique_cleaned_columns)}\n")

# Imprime em formato de lista (mais fácil de visualizar)
for col in unique_cleaned_columns:
    print(col)

print("\n--- FIM DA INSPEÇÃO ---")

--- Iniciando a inspeção das colunas para 10 tickers ---

--- Nomes ÚNICOS e LIMPOS das Colunas do YFinance Encontradas: ---
Total de itens financeiros únicos: 270

AccountsPayable
AccountsReceivable
AccumulatedDepreciation
AdditionalPaidInCapital
AllowanceForDoubtfulAccountsReceivable
AmortizationCashFlow
AmortizationOfIntangibles
AssetImpairmentCharge
AssetsHeldForSaleCurrent
AvailableForSaleSecurities
AverageDilutionEarnings
BasicAverageShares
BasicEPS
BeginningCashPosition
BuildingsAndImprovements
CapitalExpenditure
CapitalLeaseObligations
CapitalStock
CashAndCashEquivalents
CashCashEquivalentsAndFederalFundsSold
CashCashEquivalentsAndShortTermInvestments
CashDividendsPaid
CashEquivalents
CashFinancial
CashFlowFromContinuingFinancingActivities
CashFlowFromContinuingInvestingActivities
CashFlowFromContinuingOperatingActivities
ChangeInAccountPayable
ChangeInAccruedExpense
ChangeInIncomeTaxPayable
ChangeInInventory
ChangeInOtherCurrentAssets
ChangeInOtherCurrentLiabilities
ChangeInOt

In [10]:
# --- 2. Coleta de Dados Anuais (YFinance) ---

START_YEAR = 2020 # Define o ano mínimo de início dos dados
df_final = pd.DataFrame()

print(f"\n--- 2. Coletando dados anuais de {len(tickers)} tickers (Anos: {START_YEAR} em diante) ---")

for i, ticker in enumerate(tickers):
    print(f"[{i+1}/{len(tickers)}] Coletando {ticker}...")
    try:
        stock = yf.Ticker(ticker)

        # 2.1. Coletar e Concatenar Demonstrações Financeiras Anuais
        bs = stock.balance_sheet.T # Balanço Patrimonial
        is_ = stock.financials.T     # Demonstração do Resultado
        cf = stock.cashflow.T        # Fluxo de Caixa

        # Coletar info para Setor/Indústria
        info = stock.info
        
        # Concatena as demonstrações, usando a data fiscal como índice
        df_statements = pd.concat([bs, is_, cf], axis=1)
        df_statements['ticker'] = ticker
        df_statements['Sector'] = info.get('sector', 'N/A')
        df_statements['Industry'] = info.get('industry', 'N/A')
        df_statements['SharesOutstanding'] = info.get('sharesOutstanding', np.nan)


        # 2.2. Coletar o Preço Anual de Fechamento (Close Price)
        df_statements['ClosePrice'] = np.nan

        for idx, row in df_statements.iterrows():
            end_date_fiscal = idx.strftime('%Y-%m-%d')
            
            # Busca o preço nos dias após o final do ano fiscal (evita feriados)
            price_hist = stock.history(start=end_date_fiscal, end=pd.to_datetime(end_date_fiscal) + pd.Timedelta(days=5))

            if not price_hist.empty:
                # Usa o primeiro preço de fechamento encontrado
                price = price_hist['Close'].iloc[0]
                df_statements.loc[idx, 'ClosePrice'] = price
        
        # Adiciona a data como uma coluna 'date'
        df_statements.reset_index(inplace=True)
        df_statements.rename(columns={'index': 'date'}, inplace=True)
        df_statements['year'] = df_statements['date'].dt.year

        # Filtra anos a partir de START_YEAR
        df_statements = df_statements[df_statements['year'] >= START_YEAR]

        # 2.3. Adiciona ao DataFrame Final
        df_final = pd.concat([df_final, df_statements], ignore_index=True)

    except Exception as e:
        print(f"❌ Falha ao coletar dados para {ticker}: {e}")
    
    time.sleep(0.5) # Pequeno delay para ser gentil com o servidor

# Limpar o nome das colunas: remove espaços, barras e caracteres especiais
df_final.columns = df_final.columns.str.replace(r'[^A-Za-z0-9_]+', '', regex=True)

# Salvar dados brutos coletados (opcional)
df_final.to_csv("yfinance_statements_raw.csv", index=False)
print("\n✅ Coleta de dados YFinance concluída e salva em 'yfinance_statements_raw.csv'.")


--- 2. Coletando dados anuais de 637 tickers (Anos: 2020 em diante) ---
[1/637] Coletando A...
[2/637] Coletando AA...
[3/637] Coletando AAL...
[4/637] Coletando AAP...
[5/637] Coletando AAPL...
[6/637] Coletando ABBV...
[7/637] Coletando ABNB...
[8/637] Coletando ABT...
[9/637] Coletando ACGL...
[10/637] Coletando ACN...
[11/637] Coletando ACT...


$ACT: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-01-05 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1609822800")


[12/637] Coletando ADBE...
[13/637] Coletando ADCT...
[14/637] Coletando ADI...
[15/637] Coletando ADM...
[16/637] Coletando ADP...
[17/637] Coletando ADSK...
[18/637] Coletando ADT...
[19/637] Coletando AEE...
[20/637] Coletando AEP...
[21/637] Coletando AES...
[22/637] Coletando AFL...
[23/637] Coletando AIG...
[24/637] Coletando AIV...
[25/637] Coletando AIZ...
[26/637] Coletando AJG...
[27/637] Coletando AKAM...
[28/637] Coletando ALB...
[29/637] Coletando ALGN...
[30/637] Coletando ALK...
[31/637] Coletando ALL...
[32/637] Coletando ALLE...
[33/637] Coletando AMAT...
[34/637] Coletando AMCR...
[35/637] Coletando AMD...
[36/637] Coletando AME...
[37/637] Coletando AMG...
[38/637] Coletando AMGN...
[39/637] Coletando AMP...
[40/637] Coletando AMT...
[41/637] Coletando AMTM...


$AMTM: possibly delisted; no price data found  (1d 2021-09-30 -> 2021-10-05 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1632974400, endDate = 1633406400")
$AMTM: possibly delisted; no price data found  (1d 2022-09-30 -> 2022-10-05 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1664510400, endDate = 1664942400")
$AMTM: possibly delisted; no price data found  (1d 2023-09-30 -> 2023-10-05 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1696046400, endDate = 1696478400")


[42/637] Coletando AMZN...
[43/637] Coletando AN...
[44/637] Coletando ANET...
[45/637] Coletando ANF...
[46/637] Coletando AON...
[47/637] Coletando AOS...
[48/637] Coletando APA...
[49/637] Coletando APD...
[50/637] Coletando APH...
[51/637] Coletando APO...
[52/637] Coletando APP...


...

[531/637] Coletando STLD...
[532/637] Coletando STT...
[533/637] Coletando STX...
[534/637] Coletando STZ...
[535/637] Coletando SUN...
[536/637] Coletando SW...
[537/637] Coletando SWK...
[538/637] Coletando SWKS...
[539/637] Coletando SYF...
[540/637] Coletando SYK...
[541/637] Coletando SYY...
[542/637] Coletando T...
[543/637] Coletando TAP...
[544/637] Coletando TDC...
[545/637] Coletando TDG...
[546/637] Coletando TDY...
[547/637] Coletando TE...
[548/637] Coletando TECH...
[549/637] Coletando TEL...
[550/637] Coletando TER...
[551/637] Coletando TFC...
[552/637] Coletando TFX...
[553/637] Coletando TGNA...
[554/637] Coletando TGT...
[555/637] Coletando THC...
[556/637] Coletando TJX...
[557/637] Coletando TKO...
[558/637] Coletando TMC...


$TMC: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-01-05 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1609822800")


[559/637] Coletando TMO...
[560/637] Coletando TMUS...
[561/637] Coletando TPL...
[562/637] Coletando TPR...
[563/637] Coletando TRGP...
[564/637] Coletando TRIP...
[565/637] Coletando TRMB...
[566/637] Coletando TROW...
[567/637] Coletando TRV...
[568/637] Coletando TSCO...
[569/637] Coletando TSLA...
[570/637] Coletando TSN...
[571/637] Coletando TT...
[572/637] Coletando TTD...
[573/637] Coletando TTWO...
[574/637] Coletando TXN...
[575/637] Coletando TXT...
[576/637] Coletando TYL...
[577/637] Coletando UA...
[578/637] Coletando UAA...
[579/637] Coletando UAL...
[580/637] Coletando UBER...
[581/637] Coletando UDR...
[582/637] Coletando UHS...
[583/637] Coletando ULTA...
[584/637] Coletando UNH...
[585/637] Coletando UNM...
[586/637] Coletando UNP...
[587/637] Coletando UPS...
[588/637] Coletando URBN...
[589/637] Coletando URI...
[590/637] Coletando USB...
[591/637] Coletando Uber...
[592/637] Coletando V...
[593/637] Coletando VFC...
[594/637] Coletando VICI...
[595/637] Coletando

$VLTO: possibly delisted; no price data found  (1d 2020-12-31 -> 2021-01-05 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1609390800, endDate = 1609822800")
$VLTO: possibly delisted; no price data found  (1d 2021-12-31 -> 2022-01-05 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1640926800, endDate = 1641358800")
$VLTO: possibly delisted; no price data found  (1d 2022-12-31 -> 2023-01-05 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1672462800, endDate = 1672894800")


[597/637] Coletando VMC...
[598/637] Coletando VNO...
[599/637] Coletando VNT...
[600/637] Coletando VRSK...
[601/637] Coletando VRSN...
[602/637] Coletando VRTX...
[603/637] Coletando VST...
[604/637] Coletando VTR...
[605/637] Coletando VTRS...
[606/637] Coletando VZ...
[607/637] Coletando WAB...
[608/637] Coletando WAT...
[609/637] Coletando WB...
[610/637] Coletando WBD...
[611/637] Coletando WDAY...
[612/637] Coletando WDC...
[613/637] Coletando WEC...
[614/637] Coletando WELL...
[615/637] Coletando WFC...
[616/637] Coletando WHR...
[617/637] Coletando WM...
[618/637] Coletando WMB...
[619/637] Coletando WMT...
[620/637] Coletando WRB...
[621/637] Coletando WSM...
[622/637] Coletando WST...
[623/637] Coletando WTW...
[624/637] Coletando WU...
[625/637] Coletando WY...
[626/637] Coletando WYNN...
[627/637] Coletando XEL...
[628/637] Coletando XOM...
[629/637] Coletando XRAY...
[630/637] Coletando XRX...
[631/637] Coletando XYL...
[632/637] Coletando XYZ...
[633/637] Coletando YUM..

In [3]:
print("\n--- Bloco 2.5: Preenchimento Focado em PricePrevYear ---")

# 1. Carregar o arquivo intermediário
try:
    df_enrich = pd.read_csv("yfinance_statements_raw.csv")
except FileNotFoundError:
    print("❌ Erro: O arquivo 'yfinance_statements_raw.csv' não foi encontrado. Execute o Bloco 2 (Coleta de Dados) primeiro.")
    exit()

# 2. Conversão e Limpeza Essencial
# Converte o timestamp para datetime (timezone-naive)
df_enrich['date'] = pd.to_datetime(df_enrich['date'], unit='ns') 

# Garante que a coluna PricePrevYear exista (caso tenha sido excluída)
if 'PricePrevYear' not in df_enrich.columns:
    df_enrich['PricePrevYear'] = np.nan

# Garante que as colunas críticas sejam string ou numérica, e filtra tickers válidos
df_enrich['ticker'] = df_enrich['ticker'].astype(str)
df_enrich.dropna(subset=['ticker'], inplace=True)


# 3. Identifica tickers e datas onde o PricePrevYear está faltando
# Considera apenas linhas onde a data do ano fiscal está disponível
missing_prices = df_enrich[df_enrich['PricePrevYear'].isna() & df_enrich['date'].notna()]
unique_missing_tickers = missing_prices['ticker'].unique()

print(f"Total de {len(missing_prices)} linhas com PricePrevYear faltando.")

if len(unique_missing_tickers) > 0:
    print(f"🧠 Buscando preços ausentes para {len(unique_missing_tickers)} tickers...")
    
    # Itera apenas sobre os tickers onde o preço anterior está faltando
    for i, ticker in enumerate(unique_missing_tickers):
        print(f"[{i+1}/{len(unique_missing_tickers)}] Buscando histórico de preços para {ticker}...")
        
        try:
            stock = yf.Ticker(ticker)
            price_hist = stock.history(period="max", interval="1d", auto_adjust=True)['Close']
            
            # CORREÇÃO DE FUSO HORÁRIO: Remove a informação de fuso horário para comparação
            if price_hist.index.tz is not None:
                price_hist.index = price_hist.index.tz_localize(None) 
            
            # Filtra as linhas faltantes para o ticker atual
            ticker_rows_indices = missing_prices[missing_prices['ticker'] == ticker].index
            
            for index in ticker_rows_indices:
                end_date = df_enrich.loc[index, 'date']
                
                # Lógica de Busca de Preço: 1 ano antes
                prev_date_limit = end_date - pd.Timedelta(days=365)
                
                # Busca o último preço ANTES ou IGUAL ao limite de 1 ano
                price_start_check = price_hist[price_hist.index <= prev_date_limit].tail(1)
                
                if not price_start_check.empty:
                    # Atualiza o DataFrame principal
                    df_enrich.loc[index, 'PricePrevYear'] = price_start_check.iloc[0]
                
            time.sleep(0.1) # Delay para evitar bloqueio da API
            
        except Exception as e:
            print(f"⚠️ Aviso: Falha na busca de preço para {ticker}: {e}")

# 4. Salvar o DataFrame enriquecido
df_enrich.to_csv("yfinance_statements_raw_com_preco.csv", index=False)

print("\n✅ PricePrevYear preenchido e arquivo 'yfinance_statements_raw_com_preco.csv' atualizado!")

# Próximo Passo: Rodar o Bloco 3 (Cálculo dos Indicadores)


--- Bloco 2.5: Preenchimento Focado em PricePrevYear ---
Total de 3146 linhas com PricePrevYear faltando.
🧠 Buscando preços ausentes para 637 tickers...
[1/637] Buscando histórico de preços para A...
[2/637] Buscando histórico de preços para AA...
[3/637] Buscando histórico de preços para AAL...
[4/637] Buscando histórico de preços para AAP...
[5/637] Buscando histórico de preços para AAPL...
[6/637] Buscando histórico de preços para ABBV...
[7/637] Buscando histórico de preços para ABNB...
[8/637] Buscando histórico de preços para ABT...
[9/637] Buscando histórico de preços para ACGL...
[10/637] Buscando histórico de preços para ACN...
[11/637] Buscando histórico de preços para ACT...
[12/637] Buscando histórico de preços para ADBE...
[13/637] Buscando histórico de preços para ADCT...
[14/637] Buscando histórico de preços para ADI...
[15/637] Buscando histórico de preços para ADM...
[16/637] Buscando histórico de preços para ADP...
[17/637] Buscando histórico de preços para ADSK...
[

In [6]:
# --- 3. Processamento e Cálculo dos 10 Indicadores Fundamentais + Retorno ---

print("\n--- 3. Bloco Final: Processando Indicadores (Correção de Duplicatas Aplicada) ---")

# 1. Carregar o arquivo e TRATAMENTO ESSENCIAL
df_indicators = pd.read_csv("yfinance_statements_raw_com_preco.csv")

# 🚨 CORREÇÃO DE DUPLICATAS: Remove linhas repetidas no índice
# (O índice padrão do Pandas é o ID da linha, que pode estar duplicado após concatenações)
df_indicators.drop_duplicates(inplace=True)
print(f"Linhas após remoção de duplicatas: {len(df_indicators)}")

# 🚨 CORREÇÃO 1: Tratar a coluna 'date'
df_indicators['date'] = pd.to_datetime(df_indicators['date'], unit='ns') 
df_indicators['year'] = df_indicators['date'].dt.year

# 🚨 CORREÇÃO 2: Limpeza e Garantia dos Dados de Identificação
print(f"Linhas antes da limpeza de identificadores: {len(df_indicators)}")
df_indicators.dropna(subset=['ticker', 'Sector', 'Industry'], inplace=True)
print(f"Linhas após a limpeza de identificadores: {len(df_indicators)}")

# --- RESILIÊNCIA CONTRA KEYERRORS e Recálculo ---
# O restante do bloco foca nos cálculos e deve funcionar após a unicidade do índice.
# ... (O restante do código, inalterado)
# [Removido o loop de busca de preço yf.Ticker, pois agora ele está no Bloco 2.5]
# ...

# Mapeamento e Conversão para Numérico
COLUMN_MAPPING = {
    'NetIncome': 'NetIncomeLoss', 'TotalRevenue': 'Revenues', 'GrossProfit': 'GrossProfit',
    'DilutedEPS': 'EarningsPerShareDiluted', 'TotalAssets': 'Assets',
    'TotalStockholdersEquity': 'StockholdersEquity', 'TotalDebt': 'TotalDebt', 
    'CurrentAssets': 'AssetsCurrent', 'CurrentLiabilities': 'LiabilitiesCurrent',
    'TotalCashFromOperatingActivities': 'NetCashProvidedByUsedInOperatingActivities',
    'CapitalExpenditures': 'CapitalExpenditures', 
    'CashDividendsPaid': 'PaymentsOfDividendsCommonStock'
}

df_indicators = df_indicators.rename(columns={k: v for k, v in COLUMN_MAPPING.items() if k in df_indicators.columns})

final_calc_cols = list(COLUMN_MAPPING.values()) + ['ClosePrice', 'PricePrevYear', 'SharesOutstanding', 'NetIncomeLoss', 'Revenues']
for col in final_calc_cols:
    if col not in df_indicators.columns:
        df_indicators[col] = np.nan
    try:
        df_indicators[col] = pd.to_numeric(df_indicators[col], errors='coerce')
    except Exception as e:
        print(f"⚠️ Aviso: Falha ao converter '{col}' para numérica. Ignorando a coluna: {e}")
        pass

# --- CÁLCULO FINAL DOS INDICADORES E RETORNO ---
# Esta seção agora deve funcionar sem o ValueError
df_indicators["Return_Price"] = (df_indicators["ClosePrice"] - df_indicators["PricePrevYear"]) / df_indicators["PricePrevYear"]
df_indicators["Dividends_Per_Share"] = abs(df_indicators["PaymentsOfDividendsCommonStock"]) / df_indicators["SharesOutstanding"]
df_indicators["Return_Total"] = (df_indicators["ClosePrice"] - df_indicators["PricePrevYear"] + df_indicators["Dividends_Per_Share"]) / df_indicators["PricePrevYear"]

# ... (Restante dos cálculos de indicadores)
df_indicators["ROE"] = df_indicators["NetIncomeLoss"] / df_indicators["StockholdersEquity"] 
df_indicators["NetMargin"] = df_indicators["NetIncomeLoss"] / df_indicators["Revenues"] 
df_indicators["GrossMargin"] = df_indicators["GrossProfit"] / df_indicators["Revenues"]
df_indicators["ROA"] = df_indicators["NetIncomeLoss"] / df_indicators["Assets"] 
df_indicators["CurrentRatio"] = df_indicators["AssetsCurrent"] / df_indicators["LiabilitiesCurrent"]
df_indicators["DebtToEquity"] = df_indicators["TotalDebt"] / df_indicators["StockholdersEquity"]
df_indicators["MarketCap"] = df_indicators["ClosePrice"] * df_indicators["SharesOutstanding"]
df_indicators["PE"] = df_indicators["ClosePrice"] / df_indicators["EarningsPerShareDiluted"]
df_indicators.loc[df_indicators["EarningsPerShareDiluted"] <= 0, "PE"] = np.nan
df_indicators["FCF"] = df_indicators["NetCashProvidedByUsedInOperatingActivities"] + df_indicators["CapitalExpenditures"] 
df_indicators["FCF_Yield"] = df_indicators["FCF"] / df_indicators["MarketCap"] 
df_indicators["DividendPayout"] = abs(df_indicators["PaymentsOfDividendsCommonStock"]) / df_indicators["NetIncomeLoss"] 
df_indicators.loc[df_indicators["NetIncomeLoss"] <= 0, "DividendPayout"] = np.nan


# --- Seleção Final de Colunas e Salvar ---
final_cols = ['ticker', 'year', 'date', 'Sector', 'Industry', 
              "Return_Total", "Return_Price", "ROE", "NetMargin", "GrossMargin", "DebtToEquity", 
              "CurrentRatio", "ROA", "PE", "FCF_Yield", "MarketCap", "DividendPayout", 
              "ClosePrice", "PricePrevYear"]

df_final_indicators = df_indicators[[col for col in final_cols if col in df_indicators.columns]].copy()

df_final_indicators.to_csv("yfinance_final_indicators_com_retorno_total.csv", index=False)
print("✅ Bloco 3 Concluído. O Retorno Total (incluindo dividendos) está calculado!")

# Exibe o cabeçalho
print("\nPrimeiras 10 linhas da tabela de indicadores:")
preview_cols = ['ticker', 'year', 'date', 'Sector', 'Return_Total', 'ROE', 'NetMargin', 'PE']
print(df_final_indicators[preview_cols].head(10).to_markdown(index=False, floatfmt=".4f"))


--- 3. Bloco Final: Processando Indicadores (Correção de Duplicatas Aplicada) ---
Linhas após remoção de duplicatas: 3146
Linhas antes da limpeza de identificadores: 3146
Linhas após a limpeza de identificadores: 3141
✅ Bloco 3 Concluído. O Retorno Total (incluindo dividendos) está calculado!

Primeiras 10 linhas da tabela de indicadores:
| ticker   |   year | date                | Sector          |   Return_Total |      ROE |   NetMargin |       PE |
|:---------|-------:|:--------------------|:----------------|---------------:|---------:|------------:|---------:|
| A        |   2020 | 2020-10-31 00:00:00 | Healthcare      |       nan      | nan      |    nan      | nan      |
| A        |   2021 | 2021-10-31 00:00:00 | Healthcare      |         0.5557 |   0.2245 |      0.1915 |  38.7522 |
| A        |   2022 | 2022-10-31 00:00:00 | Healthcare      |        -0.1101 |   0.2364 |      0.1831 |  32.3819 |
| A        |   2023 | 2023-10-31 00:00:00 | Healthcare      |        -0.2407 |   0.